In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 사용자가 평점을 부여한 영화에 한해서 예측 성능 평가를 함
def get_evaluation(prd, actual):
    # ignore nonzero
    prd = prd[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(prd, actual), r2_score(prd, actual)

In [3]:
# 아이템 기반의 협업 필터링에서 개인화된 예측 평점 도출 공식
def predict_rating(ratings_arr, item_sim_arr):
    ratings_prd = ratings_arr.dot(item_sim_arr) / np.sum(np.abs(item_sim_arr))
    return ratings_prd

In [17]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 예측 행렬 0으로 초기화
    prd = np.zeros(ratings_arr.shape)
    print("### predict_rating_topsim - ratings_arr ###\n", ratings_arr, "\n", ratings_arr.shape)
    print("### predict_rating_topsim - item_sim_arr ###\n", item_sim_arr, "\n", item_sim_arr.shape)

    # 사용자-아이템 평점 행렬 열 크기만큼 반복
    for col in range(ratings_arr.shape[1]):
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 평점 계산
        for row in range(ratings_arr.shape[0]):
            ratings_prd = predict_rating(ratings_arr[row, :][top_n_items].T, item_sim_arr[col, :][top_n_items])
            prd[row, col] = ratings_prd
        if col % 10 == 0:
            print("# predict_rating_topsim:", col, "/", ratings_arr.shape[1])
    return prd

In [5]:
def get_unseen_movies(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId, :]
    already_seen = user_rating[user_rating > 0].index.tolist()
    movies_list = ratings_matrix.columns.tolist()
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    return unseen_list

In [6]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [9]:
# movies 는 movieId, title, genres 를 column 으로 가지고 있음
movies = pd.read_csv('dataset/netflix/Netflix_Dataset_Movie.csv')
# ratings 는 userId, movieId, rating, timestamp 를 column 으로 가지고 있음
ratings = pd.read_csv('dataset/netflix/Netflix_Dataset_Rating.csv')

ratings = ratings[['User_ID', 'Movie_ID', 'Rating']]

# # ratings_matrix 는 movieId에 대해 userId가 평가한 이력을 담은 pandas DataFrame
# ratings_matrixee = ratings.pivot_table("rating", index="userId", columns="movieId")
# print(ratings_matrix.head())

# 영화 제목에 대한 유저들의 평가를 적어둔 행렬 반환을 위해 merge (JOIN) 작업 수행
rating_movies = pd.merge(ratings, movies, on='Movie_ID')

# userId와 영화 제목을 pivot 해서 추천 예측을 위한 행렬 생성. NaN (null) 값은 0으로 채움.
ratings_matrix = rating_movies.pivot_table('Rating', index='User_ID', columns='Name')
ratings_matrix = ratings_matrix.fillna(0)

In [10]:
ratings_matrix

Name,10,10 Things I Hate About You,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,18 Again,1984,2 Fast 2 Furious,200 Cigarettes,2010: The Year We Make Contact,...,Xena: Warrior Princess: Season 3,Xena: Warrior Princess: Series Finale,Y Tu Mama Tambien,Yellow Submarine,Yi Yi,Yojimbo,Young Black Stallion,Youngblood,Yu-Gi-Oh!: The Movie,Zorro
User_ID,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2649378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2649388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 아이템 기반으로 추출하기 위해 transpose 적용 (적용 결과 각 행에는 title, 열에는 user의 평가가 들어가 있음)
# userId    1   2   3...
# title     ...
# title1    ...
# title2    ...
# title3    ...
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

User_ID,6,7,79,97,134,169,183,188,195,199,...,2649308,2649328,2649331,2649335,2649336,2649370,2649378,2649388,2649426,2649429
Name,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians II: Patch's London Adventure,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13 Ghosts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 각 영화를 기준으로 유저의 평가 양상이 얼마나 유사한지 cosine similarity 에 넣어서 평가
# (n, n)의 정사각 행렬 반환 (대각 행렬은 자기 자신에 대한 유사도, 1)
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
# cosine similarity 행렬에 제목을 붙여서 반환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

In [13]:
# extract similar with Spirited Away
print(item_sim_df["Spirited Away"].sort_values(ascending=False)[:10])

Name
Spirited Away                                             1.000000
Princess Mononoke                                         0.608491
Lord of the Rings: The Fellowship of the Ring             0.404676
Ghost in the Shell                                        0.397631
Being John Malkovich                                      0.393273
Nausicaa of the Valley of the Wind                        0.388427
Eternal Sunshine of the Spotless Mind                     0.378894
American Beauty                                           0.378695
Finding Nemo (Widescreen)                                 0.375678
Pirates of the Caribbean: The Curse of the Black Pearl    0.374592
Name: Spirited Away, dtype: float64


In [18]:
import warnings
warnings.filterwarnings("ignore")

ratings_prd = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)

### predict_rating_topsim - ratings_arr ###
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 3. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
 (143458, 1342)
### predict_rating_topsim - item_sim_arr ###
 [[1.         0.12103235 0.0977181  ... 0.15688489 0.02791243 0.14855605]
 [0.12103235 1.         0.17488531 ... 0.15037328 0.06582096 0.09072585]
 [0.0977181  0.17488531 1.         ... 0.10128843 0.09743653 0.07258661]
 ...
 [0.15688489 0.15037328 0.10128843 ... 1.         0.0378071  0.1029814 ]
 [0.02791243 0.06582096 0.09743653 ... 0.0378071  1.         0.03133808]
 [0.14855605 0.09072585 0.07258661 ... 0.1029814  0.03133808 1.        ]] 
 (1342, 1342)
# predict_rating_topsim: 0 / 1342
# predict_rating_topsim: 10 / 1342
# predict_rating_topsim: 20 / 1342
# predict_rating_topsim: 30 / 1342
# predict_rating_topsim: 40 / 1342
# predict_rating_topsim: 50 / 1342
# predict_rating_topsim: 60 / 1342
# predict_rating_topsim: 70 / 134

In [19]:
mse, r2 = get_evaluation(ratings_prd, ratings_matrix.values)
print("$$ Evaluation $$\n - MSE: {}\n - R2 Score: {}".format(mse, r2))
ratings_prd_matrix = pd.DataFrame(data=ratings_prd, index=ratings_matrix.index, columns=ratings_matrix.columns)

$$ Evaluation $$
 - MSE: 2.381904635178621
 - R2 Score: -1.8463608997796022


In [20]:
ratings_matrix

Name,10,10 Things I Hate About You,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,18 Again,1984,2 Fast 2 Furious,200 Cigarettes,2010: The Year We Make Contact,...,Xena: Warrior Princess: Season 3,Xena: Warrior Princess: Series Finale,Y Tu Mama Tambien,Yellow Submarine,Yi Yi,Yojimbo,Young Black Stallion,Youngblood,Yu-Gi-Oh!: The Movie,Zorro
User_ID,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2649378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2649388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
userNum = int(input())
user_rating_id = ratings_matrix.loc[userNum, :]
print("유저", userNum, "의 평가 내용:")
user_rating_id[user_rating_id > 0].sort_values(ascending=False)

유저 6 의 평가 내용:


Name
Ray                                              5.0
Braveheart                                       5.0
The Godfather                                    5.0
Lord of the Rings: The Fellowship of the Ring    5.0
Signs                                            5.0
                                                ... 
X2: X-Men United                                 2.0
Napoleon Dynamite                                1.0
Six Feet Under: Season 4                         1.0
Curb Your Enthusiasm: Season 3                   1.0
S.W.A.T.                                         1.0
Name: 6, Length: 144, dtype: float64

In [23]:
unseen_list = get_unseen_movies(ratings_matrix, userNum)
recommend_movies = recomm_movie_by_userid(ratings_prd_matrix, userNum, unseen_list, top_n=10)
recommend_movies = pd.DataFrame(data=recommend_movies.values, index=recommend_movies.index, columns=['prd_score'])

In [24]:
print("최종 추천 결과:", recommend_movies)

최종 추천 결과:                                 prd_score
Name                                     
The Ladykillers                  3.534967
Speed                            3.209120
Training Day                     3.053509
Secondhand Lions                 3.005262
Mean Girls                       2.964602
The Missing                      2.892093
Daredevil                        2.874906
The Lion King: Special Edition   2.816981
House of Sand and Fog            2.755288
Patch Adams                      2.743082
